# Librairies

In [98]:
import re
import pandas as pd

from bs4 import BeautifulSoup

pd.set_option('display.max_colwidth', None)

# Chargement des données

In [2]:
with open("ca_cib_deu_2022.html") as fp:
    soup = BeautifulSoup(fp, 'html.parser')

## Exploration des tags

In [74]:
set(tag.name for tag in soup.find_all())

{'a',
 'body',
 'br',
 'button',
 'deepl-input-controller',
 'div',
 'head',
 'html',
 'image',
 'img',
 'input',
 'ix:continuation',
 'ix:footnote',
 'ix:header',
 'ix:nonfraction',
 'ix:nonnumeric',
 'ix:references',
 'ix:relationship',
 'ix:resources',
 'label',
 'li',
 'link',
 'link:schemaref',
 'meta',
 'option',
 'p',
 'path',
 'script',
 'select',
 'span',
 'style',
 'svg',
 'table',
 'tbody',
 'td',
 'template',
 'th',
 'title',
 'tr',
 'ul',
 'xbrldi:explicitmember',
 'xbrli:context',
 'xbrli:divide',
 'xbrli:enddate',
 'xbrli:entity',
 'xbrli:identifier',
 'xbrli:instant',
 'xbrli:measure',
 'xbrli:period',
 'xbrli:scenario',
 'xbrli:startdate',
 'xbrli:unit',
 'xbrli:unitdenominator',
 'xbrli:unitnumerator'}

### Header XBLR

In [37]:
soup.find_all('ix:header')

[<ix:header>
 <ix:references xml:lang="fr">
 <link:schemaref xlink:href="https://www.ca-cib.fr/1VUV7VQFKUOQSJ21A208-2022-12-31.xsd" xlink:type="simple"></link:schemaref>
 </ix:references>
 <ix:resources>
 <xbrli:context id="C03814190009">
 <xbrli:entity>
 <xbrli:identifier scheme="http://standards.iso.org/iso/17442">1VUV7VQFKUOQSJ21A208</xbrli:identifier>
 </xbrli:entity>
 <xbrli:period>
 <xbrli:startdate>2022-01-01</xbrli:startdate>
 <xbrli:enddate>2022-12-31</xbrli:enddate>
 </xbrli:period>
 </xbrli:context>
 <xbrli:context id="C02039182647">
 <xbrli:entity>
 <xbrli:identifier scheme="http://standards.iso.org/iso/17442">1VUV7VQFKUOQSJ21A208</xbrli:identifier>
 </xbrli:entity>
 <xbrli:period>
 <xbrli:startdate>2021-01-01</xbrli:startdate>
 <xbrli:enddate>2021-12-31</xbrli:enddate>
 </xbrli:period>
 </xbrli:context>
 <xbrli:context id="C01907571707">
 <xbrli:entity>
 <xbrli:identifier scheme="http://standards.iso.org/iso/17442">1VUV7VQFKUOQSJ21A208</xbrli:identifier>
 </xbrli:entity>
 

In [147]:
xbrli_contexts = soup.find_all('xbrli:context')

xbrli_contexts[10]

<xbrli:context id="C0-1196768561794403669132564345">
<xbrli:entity>
<xbrli:identifier scheme="http://standards.iso.org/iso/17442">1VUV7VQFKUOQSJ21A208</xbrli:identifier>
</xbrli:entity>
<xbrli:period>
<xbrli:instant>2020-12-31</xbrli:instant>
</xbrli:period>
<xbrli:scenario>
<xbrldi:explicitmember dimension="ifrs-full:ComponentsOfEquityAxis">CACIB_2022_FR:OtherComprehensiveIncomeOnItemsThatWillNotBeReclassifiedToProfitOrLossAttributableToOwnersOfParentMember</xbrldi:explicitmember>
<xbrldi:explicitmember dimension="ifrs-full:RetrospectiveApplicationAndRetrospectiveRestatementAxis">ifrs-full:PreviouslyStatedMember</xbrldi:explicitmember>
</xbrli:scenario>
</xbrli:context>

In [156]:
xbrli_contexts[10].find_all('xbrldi:explicitmember')[0].attrs

{'dimension': 'ifrs-full:ComponentsOfEquityAxis'}

In [157]:
dimensions = {}

for dimension in xbrli_contexts[10].find_all('xbrldi:explicitmember'):
    dimensions[dimension.attrs['dimension']] = dimension.string
    
dimensions

{'ifrs-full:ComponentsOfEquityAxis': 'CACIB_2022_FR:OtherComprehensiveIncomeOnItemsThatWillNotBeReclassifiedToProfitOrLossAttributableToOwnersOfParentMember',
 'ifrs-full:RetrospectiveApplicationAndRetrospectiveRestatementAxis': 'ifrs-full:PreviouslyStatedMember'}

In [161]:
parsed_xbrli_contexts = []

for tag in xbrli_contexts:
    
    context = tag.attrs['id']
    
    if tag.find_all('xbrli:instant'):
        period = {
            "instant": tag.find_all('xbrli:instant')[0].string
        }
    
    else:
        period = {
            "startdate": tag.find_all('xbrli:startdate')[0].string,
            "enddate": tag.find_all('xbrli:enddate')[0].string
        }
    
    dimensions = {}
    
    for dimension in tag.find_all('xbrldi:explicitmember'):
        dimensions[dimension.attrs['dimension']] = dimension.string
        
    parsed_xbrli_contexts.append({
        "context": context,
        "period": period,
        "dimensions": dimensions
        
    })
    
    

contexts = pd.DataFrame(parsed_xbrli_contexts)

contexts.head(20)

context  \
0                       C03814190009   
1                       C02039182647   
2                       C01907571707   
3                       C01020068026   
4     C0-956718228794403669132564345   
5    C0-1620843255794403669132564345   
6     C01871298300794403669132564345   
7      C0650752354794403669132564345   
8     C01446511261794403669132564345   
9    C0-2055873773794403669132564345   
10   C0-1196768561794403669132564345   
11    C0-999834575794403669132564345   
12    C01905002399794403669132564345   
13   C0-1722429207794403669132564345   
14   C0-956718228-623437604132564345   
15  C0-1620843255-623437604132564345   
16   C01871298300-623437604132564345   
17    C0650752354-623437604132564345   
18   C01446511261-623437604132564345   
19  C0-2055873773-623437604132564345   

                                                  period  \
0   {'startdate': '2022-01-01', 'enddate': '2022-12-31'}   
1   {'startdate': '2021-01-01', 'enddate': '2021-12-31'}   
2                              {'instant': '2022-12-31'}   
3                              {'instant': '2021-12-31'}   
4                              {'instant': '2020-12-31'}   
5                              {'instant': '2020-12-31'}   
6                              {'instant': '2020-12-31'}   
7                              {'instant': '2020-12-31'}   
8                              {'instant': '2020-12-31'}   
9                              {'instant': '2020-12-31'}   
10                             {'instant': '2020-12-31'}   
11                             {'instant': '2020-12-31'}   
12                             {'instant': '2020-12-31'}   
13                             {'instant': '2020-12-31'}   
14                             {'instant': '2020-12-31'}   
15                             {'instant': '2020-12-31'}   
16                             {'instant': '2020-12-31'}   
17                             {'instant': '2020-12-31'}   
18                             {'instant': '2020-12-31'}   
19                             {'instant': '2020-12-31'}   

                                                                                                                                                                                                                                                                                       dimensions  
0                                                                                                                                                                                                                                                                                              {}  
1                                                                                                                                                                                                                                                                                              {}  
2                                                                                                                                                                                                                                                                                              {}  
3                                                                                                                                                                                                                                                                                              {}  
4                                                                                                                  {'ifrs-full:ComponentsOfEquityAxis': 'ifrs-full:IssuedCapitalMember', 'ifrs-full:RetrospectiveApplicationAndRetrospectiveRestatementAxis': 'ifrs-full:PreviouslyStatedMember'}  
5                                                                                        {'ifrs-full:ComponentsOfEquityAxis': 'CACIB_2022_FR:SharePremiumAndConsolidatedReserve

### Les informations numériques

In [94]:
soup.find_all('ix:nonfraction')[-2:]

[<ix:nonfraction contextref="C03814190009" decimals="2" format="ixt:num-comma-decimal" name="ifrs-full:DividendsProposedOrDeclaredBeforeFinancialStatementsAuthorisedForIssueButNotRecognisedAsDistributionToOwnersPerShare" scale="0" unitref="EURpershare">1,18</ix:nonfraction>,
 <ix:nonfraction contextref="C03814190009" decimals="2" format="ixt:num-comma-decimal" name="ifrs-full:DividendsProposedOrDeclaredBeforeFinancialStatementsAuthorisedForIssueButNotRecognisedAsDistributionToOwners" scale="0" unitref="EUR">343 145 588,28</ix:nonfraction>]

In [20]:
type(soup.find_all('ix:nonfraction')[1])

bs4.element.Tag

In [21]:
soup.find_all('ix:nonfraction')[1].string

'4 933'

In [29]:
ix_nonfraction = soup.find_all('ix:nonfraction')
parsed_ix_nonfraction = []

for tag in ix_nonfraction:
    parsed_tag = {
        "ID": tag["contextref"],
        "decimals": tag["decimals"],
        "name": tag["name"],
        "format": tag["format"],
        "scale": tag["scale"],
        "value": tag.string
    }
    
    parsed_ix_nonfraction.append(parsed_tag)

In [99]:
kpi = pd.DataFrame(parsed_ix_nonfraction)
kpi['nomenclature'] = kpi['name'].apply(lambda x: x.split(':')[0])
kpi['name'] = kpi['name'].apply(lambda x: x.split(':')[1])
kpi

ID decimals  \
0    C03814190009       -6   
1    C02039182647       -6   
2    C03814190009       -6   
3    C02039182647       -6   
4    C03814190009       -6   
..            ...      ...   
889  C03814190009       -6   
890  C03814190009       -6   
891  C03814190009        2   
892  C03814190009        2   
893  C03814190009        2   

                                                                                                                     name  \
0                                                                                                     RevenueFromInterest   
1                                                                                                     RevenueFromInterest   
2                                                                                                         InterestExpense   
3                                                                                                         InterestExpense   
4                                                                                                  FeeAndCommissionIncome   
..                                                                                                                    ...   
889                                                                                   RepaymentsOfSubordinatedLiabilities   
890                                                                            ProceedsFromIssueOfSubordinatedLiabilities   
891                                                                    DividendsRecognisedAsDistributionsToOwnersPerShare   
892  DividendsProposedOrDeclaredBeforeFinancialStatementsAuthorisedForIssueButNotRecognisedAsDistributionToOwnersPerShare   
893          DividendsProposedOrDeclaredBeforeFinancialStatementsAuthorisedForIssueButNotRecognisedAsDistributionToOwners   

                    format scale           value nomenclature  
0    ixt:num-comma-decimal     6           8 928    ifrs-full  
1    ixt:num-comma-decimal     6           4 933    ifrs-full  
2    ixt:num-comma-decimal     6           5 100    ifrs-full  
3    ixt:num-comma-decimal     6           1 556    ifrs-full  
4    ixt:num-comma-decimal     6           1 673    ifrs-full  
..                     ...   ...             ...          ...  
889  ixt:num-comma-decimal     6             491    ifrs-full  
890  ixt:num-comma-decimal     6             622    ifrs-full  
891  ixt:num-comma-decimal     0            1,90    ifrs-full  
892  ixt:num-comma-decimal     0            1,18    ifrs-full  
893  ixt:num-comma-decimal     0  343 145 588,28    ifrs-full  

[894 rows x 7 columns]

In [128]:
kpi.query('ID == "C0-1620843255794403669132564345"')

ID decimals    name                 format  \
207  C0-1620843255794403669132564345       -6  Equity  ixt:num-comma-decimal   

    scale   value nomenclature  
207     6  11 155    ifrs-full

In [100]:
kpi.tail().loc[:, ['ID', 'name', 'value']]

ID  \
889  C03814190009   
890  C03814190009   
891  C03814190009   
892  C03814190009   
893  C03814190009   

                                                                                                                     name  \
889                                                                                   RepaymentsOfSubordinatedLiabilities   
890                                                                            ProceedsFromIssueOfSubordinatedLiabilities   
891                                                                    DividendsRecognisedAsDistributionsToOwnersPerShare   
892  DividendsProposedOrDeclaredBeforeFinancialStatementsAuthorisedForIssueButNotRecognisedAsDistributionToOwnersPerShare   
893          DividendsProposedOrDeclaredBeforeFinancialStatementsAuthorisedForIssueButNotRecognisedAsDistributionToOwners   

              value  
889             491  
890             622  
891            1,90  
892            1,18  
893  343 145 588,28

In [90]:
kpi['format'].unique()

array(['ixt:num-comma-decimal', 'ixt:fixed-zero'], dtype=object)

In [91]:
kpi['nomenclature'].unique()

array(['ifrs-full', 'CACIB_2022_FR'], dtype=object)

In [101]:
kpi['name'].nunique()

157

In [92]:
kpi['ID'].nunique()

170

### Les informations textuelles

In [80]:
soup.find_all('ix:nonnumeric')[14]

<ix:nonnumeric contextref="C03814190009" escape="true" name="ifrs-full:NameOfReportingEntityOrOtherMeansOfIdentification"><span class="S7C0D SB6 S2" id="ta_250">Crédit Agricole Corporate and Investment Bank </span></ix:nonnumeric>

In [104]:
ix_nonnumeric = soup.find_all('ix:nonnumeric')
parsed_ix_nonnumeric = []

for tag in ix_nonnumeric:
    parsed_tag = {
        "ID": tag["contextref"],
        "name": tag["name"],
        "value": tag.text
    }
    
    parsed_ix_nonnumeric.append(parsed_tag)

In [108]:
texts = pd.DataFrame(parsed_ix_nonnumeric)
texts['nomenclature'] = texts['name'].apply(lambda x: x.split(':')[0])
texts['name'] = texts['name'].apply(lambda x: x.split(':')[1])
texts = texts.drop_duplicates()

texts.head(3)

ID                                                        name  \
0  C03814190009  DisclosureOfNotesAndOtherExplanatoryInformationExplanatory   
1  C03814190009                 DisclosureOfFinancialInstrumentsExplanatory   
2  C03814190009                           DisclosureOfCreditRiskExplanatory   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                